In [1]:
import pandas as pd

health_df = pd.read_csv('health_df.csv')

In [2]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34447 entries, 0 to 34446
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   location       34447 non-null  object
 1   client_gender  34447 non-null  int64 
 2   client_age     34447 non-null  int64 
 3   question       34447 non-null  object
 4   answer         34447 non-null  object
 5   category01     34447 non-null  object
 6   category02     34447 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.8+ MB


In [3]:
import joblib
from sentence_transformers import SentenceTransformer
from tqdm import tqdm  # tqdm 임포트

# tqdm과 pandas를 통합
tqdm.pandas()

# 1. 로지스틱 회귀 모델 로드
clf = joblib.load('logistic_regression_model.pkl')

# 2. S-BERT 모델 로드
model = SentenceTransformer('sentence_transformer_model')

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# 3. health_df의 'question'에 대해 임베딩 생성 및 tqdm 적용
health_df['embedding'] = health_df['question'].progress_apply(lambda x: model.encode(x))

100%|██████████| 34447/34447 [11:49<00:00, 48.52it/s]


In [5]:
from konlpy.tag import Okt

okt = Okt()
stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [6]:
def preprocess_text(text, stop_words):
    tokens = okt.morphs(text)  # 형태소 분석으로 토큰화
    filtered_tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return ' '.join(filtered_tokens)

In [7]:
from sentence_transformers import util, SentenceTransformer

def get_best_response(user_input, model, health_df, threshold=0.8):
    user_embedding = model.encode(user_input)  # 사용자 입력 임베딩
    best_similarity = 0
    best_response = None
    
    # 각 질문에 대해 S-BERT 유사도 계산
    for _, row in health_df.iterrows():
        similarity = util.cos_sim(user_embedding, row['embedding'])[0][0].item()
        if similarity > best_similarity:
            best_similarity = similarity
            best_response = row['answer']  # 가장 유사한 답변 업데이트
    
    # 임계값 이상의 유사도일 경우 답변 반환
    if best_similarity >= threshold:
        return best_response
    else:
        return "죄송하지만, 이해할 수 없습니다."

In [8]:
# 사용자 입력 필터링 (업데이트)
def filter_user_input(user_input, clf, model):
    processed_input = preprocess_text(user_input, stop_words)
    user_embedding = model.encode(processed_input)
    
    predicted_label = clf.predict([user_embedding])[0]
    
    if predicted_label == 1:
        return "경고: 모욕적인 텍스트가 감지되었습니다."
    else:
        # 정상 텍스트인 경우 최적의 질문과 답변 선택
        return get_best_response(processed_input, model, health_df)

In [9]:
# 사용자 입력 테스트
while True:
    user_input = input("사용자 입력 (종료하려면 'exit' 입력): ")
    if user_input.lower() == 'exit':
        break
    
    result = filter_user_input(user_input, clf, model)
    print(result)

대기실 위치를 안내해 드릴 테니 따라오시길 바랍니다.
경고: 모욕적인 텍스트가 감지되었습니다.
복도 끝 우측에 있습니다. 남자 화장실은 그 반대에 있으니 참고 부탁드립니다.
경고: 모욕적인 텍스트가 감지되었습니다.
